<a href="https://colab.research.google.com/github/mereilly/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 https://cloud.r-project.org/bin/linux/ubuntu bi

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Luggage_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()
print (f"Raw records count: {df.count()}")

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   40884699| R9CO86UUJCAW5|B00VGTN02Y|     786681372|Teenage Mutant Ni...|         Luggage|          3|            0|          0|   N|                Y|my review of this...|my review of this...| 2015-08-31|
|         US|   23208852|R3PR8X6QGVJ8B1|B005KIWL0E|     618251799|Kenneth Cole Reac...|         Luggage|          5|    

In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id',
                     'star_rating',
                     'helpful_votes',
                     'total_votes',
                     'vine',
                     'verified_purchase'])

vine_df.show()
print (f"Vine count: {vine_df.count()}")

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R9CO86UUJCAW5|          3|            0|          0|   N|                Y|
|R3PR8X6QGVJ8B1|          5|            0|          0|   N|                Y|
|R39BO2819ABUPF|          4|            0|          0|   N|                Y|
|R3ADL7V6EGGEEP|          4|            0|          0|   N|                Y|
|R1OXYPBPLVRMI5|          5|            0|          0|   N|                Y|
|R1WYM8Z5ATQ98O|          3|            0|          0|   N|                Y|
|R3LCIANTN1H9EC|          4|            1|          1|   N|                Y|
|R3U2M23N1P0KQ6|          5|            0|          0|   N|                Y|
|R29MB6N7HB6NZI|          1|            2|          2|   N|                Y|
| RGEQ6DGRG7DQG|          5|            0|          0|   N|     

In [ ]:
# Step 1: Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes 
# count is equal to or greater than 20 to pick reviews that are more likely to be helpful and to avoid having 
# division by zero errors later on.

filter_df = vine_df.filter(vine_df.total_votes >= 20)
filter_df.show()
print(f"Filter DF count: {filter_df.count()}")

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R11IBSD5E6HPSD|          1|           29|         31|   N|                Y|
|R2KVWAYBPWK1OV|          5|           20|         22|   N|                N|
|R1FLOE9E4ODIGR|          5|           34|         38|   N|                Y|
| R6XTEZCSCUJ4J|          5|           20|         23|   N|                Y|
|R2WVV9LQKIVMOX|          5|           23|         23|   N|                N|
| RYIU6GIB4TYT2|          5|           30|         30|   N|                N|
| RNXGKYVCJDDL7|          5|           28|         28|   N|                Y|
|R1E8V7J0K61754|          5|           18|         20|   N|                Y|
| RK52L3B5733BH|          5|           22|         22|   N|                N|
|R213PW7RSI6Z7N|          5|           48|         50|   N|     

In [ ]:
# Step 2: Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to 
# retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater 
# than 50%.

helpful_df = filter_df.filter(filter_df.helpful_votes/filter_df.total_votes >= 0.5)
helpful_df.show()
print(f"Helpful DF count: {helpful_df.count()}")

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R11IBSD5E6HPSD|          1|           29|         31|   N|                Y|
|R2KVWAYBPWK1OV|          5|           20|         22|   N|                N|
|R1FLOE9E4ODIGR|          5|           34|         38|   N|                Y|
| R6XTEZCSCUJ4J|          5|           20|         23|   N|                Y|
|R2WVV9LQKIVMOX|          5|           23|         23|   N|                N|
| RYIU6GIB4TYT2|          5|           30|         30|   N|                N|
| RNXGKYVCJDDL7|          5|           28|         28|   N|                Y|
|R1E8V7J0K61754|          5|           18|         20|   N|                Y|
| RK52L3B5733BH|          5|           22|         22|   N|                N|
|R213PW7RSI6Z7N|          5|           48|         50|   N|     

In [ ]:
# Step 3: Filter the DataFrame or table created in Step 2, and create a new DataFrame or table 
# that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.

vine_y_df = helpful_df.filter(helpful_df.vine == 'Y')
vine_y_df.show()
print(f"Vine Y DF count: {vine_y_df.count()}")


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RPJ7C4HP3BHXN|          5|           25|         29|   Y|                N|
|R3SB08XK0M7993|          4|           44|         49|   Y|                N|
|R26X1DACMCN3YF|          3|           16|         21|   Y|                N|
|R17HKKDE9ZRVOO|          4|           31|         35|   Y|                N|
|R1R7GNSCMO8U7I|          4|           24|         28|   Y|                N|
|R2T83WIPG3IX5D|          4|           72|         76|   Y|                N|
| RD006IWFCUA5A|          5|          122|        133|   Y|                N|
| REOQCBUVUPXP1|          4|           23|         27|   Y|                N|
|R2336R80ZEOKTX|          5|           63|         63|   Y|                N|
| RC1ZXE5GVL9TA|          4|           20|         23|   Y|     

In [ ]:
# Step 4: Repeat Step 3, but this time retrieve all the rows where the review was not part of 
# the Vine program (unpaid), vine == 'N'.

vine_n_df = helpful_df.filter(helpful_df.vine == 'N')
vine_n_df.show()
print(f"Vine N DF count: {vine_n_df.count()}")

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R11IBSD5E6HPSD|          1|           29|         31|   N|                Y|
|R2KVWAYBPWK1OV|          5|           20|         22|   N|                N|
|R1FLOE9E4ODIGR|          5|           34|         38|   N|                Y|
| R6XTEZCSCUJ4J|          5|           20|         23|   N|                Y|
|R2WVV9LQKIVMOX|          5|           23|         23|   N|                N|
| RYIU6GIB4TYT2|          5|           30|         30|   N|                N|
| RNXGKYVCJDDL7|          5|           28|         28|   N|                Y|
|R1E8V7J0K61754|          5|           18|         20|   N|                Y|
| RK52L3B5733BH|          5|           22|         22|   N|                N|
|R213PW7RSI6Z7N|          5|           48|         50|   N|     

In [ ]:
# Step 5: Determine the total number of reviews, the number of 5-star reviews, and the 
# percentage of 5-star reviews for the two types of review (paid vs unpaid).

vine_y_total = vine_y_df.count()
vine_n_total = vine_n_df.count()

vine_y_five_star = vine_y_df.filter(vine_y_df.star_rating == 5).count()
vine_n_five_star = vine_n_df.filter(vine_n_df.star_rating == 5).count()

vine_y_five_star_pct = (vine_y_five_star/vine_y_total) * 100
vine_n_five_star_pct = (vine_n_five_star/vine_n_total) * 100

print (f"Vine Y five star: {vine_y_five_star}, Vine Y total: {vine_y_total}, Five star pct: {vine_y_five_star_pct:.1f}")
print (f"Vine N five star: {vine_n_five_star}, Vine N total: {vine_n_total}, Five star pct: {vine_n_five_star_pct:.1f}")


Vine Y five star: 10, Vine Y total: 21, Five star pct: 47.6
Vine N five star: 3448, Vine N total: 6690, Five star pct: 51.5
